# Support Vector Machine

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.ensemble import BaggingClassifier
from sklearn import preprocessing # LabelEncoder
from sklearn.preprocessing import MinMaxScaler # Escala los datos
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
import scikitplot as skplt 
from string import ascii_uppercase 
# import seaborn as sns
import qgrid
import time
#from sklearn.externals import joblib # Para guardar el modelo
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

from scipy import stats #Para la moda

from sklearn.svm import SVC

In [4]:
def loadCSV(pathSamples, pathMatrix):
    df_loaded = pd.read_table(pathMatrix, sep=',')
    data = pd.read_table(pathSamples, sep=',')
    clin_trial_values = df_loaded.values
    
    Y = data['Eligible']
    Y = Y.astype(int)
    X = clin_trial_values[:, :]
    return X, Y

In [5]:
def classification_error(y_est, y_real):
    err = 0
    for y_e, y_r in zip(y_est, y_real):

        if y_e != y_r:
            err += 1

    return err/np.size(y_est)

In [6]:
def plot_roc(Xtest, Ytest, probs, xlabel):
    ns_probs = [0 for _ in range(len(Ytest))]
    
    probs = probs[:, 1]
    ns_auc = roc_auc_score(Ytest, ns_probs)
    auc = roc_auc_score(Ytest, probs)  

    print('No Skill: ROC AUC=%.3f' % (ns_auc))
    print('Logistic: ROC AUC=%.3f' % (auc))

    ns_fpr, ns_tpr, _ = roc_curve(Ytest, ns_probs)
    fpr, tpr, _ = roc_curve(Ytest, probs)   

    plt.plot(ns_fpr, ns_tpr, linestyle='--', label='No Skill')
    plt.plot(fpr, tpr, marker='.', label= xlabel)

    # axis labels
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    # show the legend
    plt.legend()
    # show the plot
    plt.show()

In [7]:
def model_SVC(ker, C, gam, impresion = False):
    
    if gam == 0:
        gam = 'auto'

    svc = BaggingClassifier(SVC(gamma=gam, C=float(C),  kernel=ker, decision_function_shape='ovo' , probability = True))
    tiempo_i = time.time()
    
    accuracy_list = np.zeros([4])
    accuracy_list_train = np.zeros([4])
    precision_list = np.zeros([4,2])
    recall_list = np.zeros([4,2])
    f_list = np.zeros([4,2]) 
    errores = np.zeros(4)
    list_acc_mean = []
    list_acc_train_mean = []
    list_acc_std = []
    list_acc_train_std = []
    # list_percentage = [0.05,0.10,0.15,0.20,0.25,0.3]
    list_percentage = [0.2]
    
    for i in list_percentage:
        for j in range(4):
            print(i, j)
            Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=i)
            scaler = MinMaxScaler()#Escala entre 0 y 1
            Xtrain = scaler.fit_transform(Xtrain)
            Xtest = scaler.transform(Xtest)

            print('fitting')
            svc.fit(Xtrain, Ytrain)
            pred = svc.predict(Xtest)
            pred_train = svc.predict(Xtrain)
            print('done')

            #code for calculating accuracy 
            _accuracy_ = accuracy_score(Ytest, pred, normalize=True)
            accuracy_list[j] = _accuracy_
            
            _accuracy_train_ = accuracy_score(Ytrain, pred_train, normalize=True)
            accuracy_list_train[j] = _accuracy_train_

            #code for calculating recall 
            _recalls_ = recall_score(Ytest, pred, average=None)
            recall_list[j] = _recalls_

            #code for calculating precision 
            _precisions_ = precision_score(Ytest, pred, average=None)
            precision_list[j] = _precisions_

            _f_score_ = f1_score(Ytest, pred, average=None)
            f_list[j] = _f_score_


            errores[j] = classification_error(pred, Ytest)
        
        list_acc_mean.append(np.mean(accuracy_list))
        list_acc_std.append(np.std(accuracy_list))
        list_acc_train_mean.append(np.mean(accuracy_list_train))
        list_acc_train_std.append(np.std(accuracy_list_train))
       
    
    if impresion == True:
        
        x = [10000-(i* 10000) for i in list_percentage]
        sneg = [m-s for m,s in zip(list_acc_mean,list_acc_std)]
        spos = [m+s for m,s in zip(list_acc_mean,list_acc_std)]
        
        sneg_train = [m-s for m,s in zip(list_acc_train_mean,list_acc_train_std)]
        spos_train = [m+s for m,s in zip(list_acc_train_mean,list_acc_train_std)]
        
        fig1, ax1 = plt.subplots()
        
        ax1.fill_between(x,sneg_train,spos_train,alpha=.4)
        ax1.plot(x, list_acc_train_mean, marker = 'v', label = "train")
        ax1.fill_between(x,sneg,spos,alpha=.4)
        ax1.plot(x, list_acc_mean, marker = 'p', label = "test")

        ax1.set_xlabel("Samples in train")
        ax1.set_ylabel("Accuracy")

        ax1.legend(loc="upper right", title="Learning curve", frameon=False)
        plt.show()
        
        #Curva ROC
        
        svc_probs = svc.predict_proba(Xtest)
        
        plot_roc(Xtest, Ytest, svc_probs, "SVM")
        skplt.metrics.plot_confusion_matrix(Ytest, pred, normalize=True)

    return str(np.mean(accuracy_list)), str(np.std(accuracy_list)), str(np.mean(recall_list)), str(np.std(recall_list)), str(np.mean(precision_list)), str(np.std(precision_list)), str(np.mean(f_list)), str(np.std(f_list)), str(np.mean(errores)), str(np.std(errores)), str(time.time()-tiempo_i)

# LSA 10k_1Col_NoCarEsp

In [8]:
X, Y = loadCSV("../../Dataset/10k_1Col_NoCarEsp_LSA.csv", "../../Tables/docsTopicsLSA1200.csv") #Cargar SCV

In [9]:
# Llenar tabla con diferentes hiperparámetros
pd.options.mode.chained_assignment = None

randn = np.random.randn
df_types = pd.DataFrame({
    'Kernel' : pd.Series(['linear','linear','linear','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf']),
    'C' : pd.Series([0.1,0.5,1, 0.1,0.1,0.1,0.5,0.5,0.5,1,1,1]),
    'gamma' : pd.Series([0,0,0, 0.1,0.5,1,0.1,0.5,1,0.1,0.5,1])})
df_types["Eficiencia"] = ""
df_types["Int_Eficiencia"] = ""
df_types["Sensibilidad"] = ""
df_types["Int_Sensibilidad"] = ""
df_types["Precision"] = ""
df_types["Int_Precision"] = ""
df_types["F-Score"] = ""
df_types["Int_F-Score"] = ""
df_types["Error_Prueba"] = ""
df_types["Int_error"] = ""
df_types["Tiempo de ejecución"] = ""
df_types.set_index(['Kernel','C','gamma'], inplace=True)

for n, k, o in df_types.index:
    print(n,k,o, '...')
    Acc, IntAcc, Sen, IntSen, Pre, IntPre, f, IntF, error, stdError, tiempo = model_SVC(n, k, o, impresion = False)
    print('aa')
    df_types["Eficiencia"][n,k,o] = Acc
    df_types["Int_Eficiencia"][n,k,o] = IntAcc
    df_types["Sensibilidad"][n,k,o] = Sen
    df_types["Int_Sensibilidad"][n,k,o] = IntSen
    df_types["Precision"][n,k,o] = Pre
    df_types["Int_Precision"][n,k,o] = IntPre
    df_types["F-Score"][n,k,o] = f
    df_types["Int_F-Score"][n,k,o] = IntF
    df_types["Error_Prueba"][n,k,o] = error
    df_types["Int_error"][n,k,o] = stdError
    df_types["Tiempo de ejecución"][n,k,o] = tiempo


#df_types.sort_index(inplace=True)
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget.get_changed_df()

linear 0.1 0.0 ...
svc
done
0.2 0
fit
done, pred
done
0.2 1
fit
done, pred
done
0.2 2
fit
done, pred
done
0.2 3
fit
done, pred
done
aa
linear 0.5 0.0 ...
svc
done
0.2 0
fit
done, pred
done
0.2 1
fit
done, pred
done
0.2 2
fit
done, pred
done
0.2 3
fit
done, pred
done
aa
linear 1.0 0.0 ...
svc
done
0.2 0
fit
done, pred
done
0.2 1
fit
done, pred
done
0.2 2
fit
done, pred
done
0.2 3
fit
done, pred
done
aa
rbf 0.1 0.1 ...
svc
done
0.2 0
fit
done, pred
done
0.2 1
fit
done, pred
done
0.2 2
fit
done, pred
done
0.2 3
fit
done, pred
done
aa
rbf 0.1 0.5 ...
svc
done
0.2 0
fit
done, pred
done
0.2 1
fit
done, pred
done
0.2 2
fit
done, pred
done
0.2 3
fit
done, pred
done
aa
rbf 0.1 1.0 ...
svc
done
0.2 0
fit
done, pred
done
0.2 1


/home/juan/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/juan/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


fit
done, pred
done
0.2 2
fit
done, pred
done
0.2 3


/home/juan/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/juan/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


fit
done, pred
done
aa
rbf 0.5 0.1 ...
svc
done
0.2 0
fit
done, pred
done
0.2 1
fit
done, pred
done
0.2 2
fit
done, pred
done
0.2 3
fit
done, pred
done
aa
rbf 0.5 0.5 ...
svc
done
0.2 0
fit
done, pred
done
0.2 1
fit
done, pred
done
0.2 2
fit
done, pred
done
0.2 3
fit
done, pred
done
aa
rbf 0.5 1.0 ...
svc
done
0.2 0
fit
done, pred
done
0.2 1
fit
done, pred
done
0.2 2
fit
done, pred
done
0.2 3
fit
done, pred
done
aa
rbf 1.0 0.1 ...
svc
done
0.2 0
fit
done, pred
done
0.2 1
fit
done, pred
done
0.2 2
fit
done, pred
done
0.2 3
fit
done, pred
done
aa
rbf 1.0 0.5 ...
svc
done
0.2 0
fit
done, pred
done
0.2 1
fit
done, pred
done
0.2 2
fit
done, pred
done
0.2 3
fit
done, pred
done
aa
rbf 1.0 1.0 ...
svc
done
0.2 0
fit
done, pred
done
0.2 1
fit
done, pred
done
0.2 2
fit
done, pred
done
0.2 3
fit
done, pred
done
aa


Eficiencia         Int_Eficiencia  \
Kernel C   gamma                                               
linear 0.1 0.0               0.785625   0.011359880060986559   
       0.5 0.0               0.788875   0.006278684177437193   
       1.0 0.0               0.786875   0.012476853569710605   
rbf    0.1 0.1                0.73775   0.005031152949374522   
           0.5     0.5503750000000001   0.012466830992678112   
           1.0    0.49549999999999994    0.00474341649025255   
       0.5 0.1                 0.7985  0.0072543090642734455   
           0.5                  0.676    0.00961769203083568   
           1.0                0.54525   0.002250000000000011   
       1.0 0.1                0.80125   0.004828301978956975   
           0.5     0.7462500000000001   0.012915591353089473   
           1.0                  0.653    0.01134129622221379   

                        Sensibilidad      Int_Sensibilidad  \
Kernel C   gamma                                             
linear 0.1 0.0    0.7856730129091053  0.013019991456228766   
       0.5 0.0    0.7888588485258268  0.018435022672234545   
       1.0 0.0    0.7868504105326869  0.016138452041556683   
rbf    0.1 0.1    0.7372877378020215   0.04241618871273195   
           0.5    0.5543746319697218    0.3892180311636081   
           1.0    0.5019515059301105   0.49505392569665174   
       0.5 0.1    0.7985191033645089  0.015201090667692969   
           0.5    0.6771440053013034   0.24129360282397144   
           1.0    0.5466387675140172   0.41906607395648754   
       1.0 0.1    0.8011686970845321   0.01105072993481533   
           0.5     0.746740063459278   0.12624933479247735   
           1.0    0.6540035326493077    0.2590073848240535   

                            Precision         Int_Precision  \
Kernel C   gamma                                              
linear 0.1 0.0     0.7856689524022763   0.01303478588836303   
       0.5 0.0     0.7892087570366673  0.012114167897951505   
       1.0 0.0     0.7868027880044052   0.01740763910266851   
rbf    0.1 0.1     0.7391497857453606   0.01793954989226582   
           0.5     0.6375336679145409   0.11346099405091455   
           1.0    0.42432603404836533   0.26092669020662473   
       0.5 0.1      0.798633940241593   0.01279603590716817   
           0.5     0.7304317094475835   0.11388254980529597   
           1.0      0.657076983567807    0.1342677760511924   
       1.0 0.1     0.8013187095415248   0.00688103057643204   
           0.5     0.7630240218488997   0.06852798893140159   
           1.0     0.7099231724112707   0.11060796506815394   

                              F-Score           Int_F-Score  \
Kernel C   gamma                                              
linear 0.1 0.0     0.7856213380598127  0.011391050661187837   
       0.5 0.0     0.7887987189950106  0.007424544065618756   
       1.0 0.0     0.7867096918272041  0.013753270167440775   
rbf    0.1 0.1      0.737062769769465  0.014304328520272797   
           0.5     0.4727880326548698   0.20263942388899206   
           1.0    0.33732192168021596   0.32389984837591684   
       0.5 0.1     0.7984132602964324  0.008330843377629025   
           0.5     0.6565151871967869   0.08258119663609291   
           1.0    0.44937022579763203     0.229826123746169   
       1.0 0.1     0.8011832109300261  0.005980430041850121   
           0.5     0.7423228936690307   0.03466477743513438   
           1.0     0.6285339183708156    0.0962015244252983   

                         Error_Prueba             Int_error  \
Kernel C   gamma                                              
linear 0.1 0.0               0.214375  0.011359880060986559   
       0.5 0.0               0.211125  0.006278684177437176   
       1.0 0.0               0.213125  0.012476853569710598   
rbf    0.1 0.1    0.26225000000000004  0.005031152949374532   
           0.5    0.44962500000000005   0.01246683099267812   
           1.0                 0.5045   0.004743416490

In [ ]:
# Llamar función con el mejor modelo
Acc, IntAcc, Sen, IntSen, Pre, IntPre, f, IntF, error, stdError, tiempo = model_SVC('rbf', 2, 1, impresion = True)
print('Eficiencia',Acc, ' Int_Eficiencia', IntAcc,' Sensibilidad', Sen, ' Int_Sensibilidad',IntSen,' Precision', Pre, ' Int_Precision',IntPre,' F-Score', f, ' Int_F-Score',IntF,' Error_Prueba', error,' Int_Error', stdError,' Tiempo ejecución', tiempo)